# Introduction

The signal data was obtained by reading digitized polygraphic EDF (European Data Format) files. EDF is a standard format for distributing polygraphic data and the source files record various measurements about the sleep study including the signals produced by chest and abdominal inductance belts which indicate the respective breathing patterns. The EDF files were accompanied by annotations in XML format that indicate various encodings such as sleep stage and hypypnea and apnea respiratory events.

The EDF signals are of varying frequencies depending on the equipment used to record the signals. The most common frequency observed was 200 Hz but the total range in the dataset was from 32-512Hz. The annotations were recorded in seconds.

# Filters & Pre-processing

The signal data was first resampled to 64Hz to provide a consistent frequency for the filtering operations. Next a finite impulse response band pass filter was applied to select frequencies that  lie in the physiological range for a child (0.1 - 5 Hz).

The annotations were then used to remove any sections from the signals where the individual was not sleeping. Additionally, the following steps were run on two samples, one with only the awake sections removed and a second where all respiratory events were additionally removed. This second signal provides a measure of asynchrony which ignores the respiratory events that were observed and coded.

# Windowing

To calculate the asynchrony between the abdominal and chest signals, we first compute the fundamental frequency of the entire signal. This fundamental frequency indicates the average respiratory period of the signal. We then slide a window of four times the average respiratory period over the signal, using a step size of one quarter of this period. Within this sliding window we check the signals for various conditions that indicate noise, removing them from the final asynchrony calculation.

For each window we apply three checks for validity:

1. Noisy signals - the signals were deemed noisy if the ratio of the spectral power within our band of interest to the total power was less than 0.65
2. Frequencies outside the expected range: if the fundamental frequency of this window was outside the expected physiological range for children (0.12 - 0.585 Hz or 7.2 - 35.1 breaths/min)
3. Large disparity between the abdominal and chest fundamental frequencies (a difference of greater than 50%)


# Asynchrony Computation

Within the sliding window we calculate the Hilbert transform of the two signals and then find the angle between the phases of this analytical signal. The Hilbert transform results in a complex vector that represents the instantaneous amplitude and phase of the signal.

$$  z(t) = z_r(t) + j z_i(t)  = x(t) + j HT \left[ x(t) \right] = Ae^{i\phi(t)} $$

Where $A$ is the instantaneous amplitude and $\phi$ is the instantaneous phase. The angle between the phases can then be computed as

$$ \angle \dfrac { z_1 \cdot \bar{z}_2 }{ \sqrt{z_1 \cdot \bar{z}_1 \times z_2 \cdot \bar{z}_2} }$$

Where $\bar{z}$ is the complex conjugate of $z$ and $\angle$ is the angle of a complex value: $\tan^{-1} \tfrac{x}{y}$.

The resulting angle is between negative $\pi$ and positive $\pi$ to indicate how far out of phase the two signals are. Since we do not care which signal is ahead we take the absolute value of the angle and divide by $\pi$ resulting in a measurement of the asynchrony that is between 0 and 1 where 0 is perfect synchrony and 1 is perfect asynchrony.

# Other

Instantaneous phase:

$$ \phi(t) = \angle z(t) = arctan \left[  \frac{z_i(t)}{z_r(t)}  \right]$$